# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Cargar etiquetas de entrenamiento (archivo CSV con columnas "image","count")
df = pd.read_csv('C:\\Admin\Downloads\personas\part_B_final\train_data.csv')  # ✅ Agregar ruta correcta
image_paths = ['train_images/' + fname for fname in df['image']]
counts = df['count'].values

# Función para leer y preprocesar una imagen
def load_image(path, count):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (128, 128))
    img = img / 255.0
    return img, tf.cast(count, tf.float32)

# Crear Dataset tf.data a partir de los paths y conteos
dataset = tf.data.Dataset.from_tensor_slices((image_paths, counts))
dataset = dataset.map(load_image).shuffle(500).batch(16).prefetch(tf.data.AUTOTUNE)

# ✅ MEJORAR: División de datos más robusta
train_size = int(0.8 * len(df))
# Calcular número de batches correctamente
train_batches = train_size // 16
val_batches = (len(df) - train_size) // 16

train_dataset = dataset.take(train_batches)
val_dataset = dataset.skip(train_batches).take(val_batches)

# ✅ MEJORAR: Modelo CNN más robusto
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),  # Más filtros
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),  # Capa adicional
    tf.keras.layers.GlobalAveragePooling2D(),  # Mejor que Flatten
    tf.keras.layers.Dense(128, activation='relu'),  # Más neuronas
    tf.keras.layers.Dropout(0.5),  # Prevenir overfitting
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)  # Capa de salida: un solo valor (conteo)
])

# ✅ MEJORAR: Configuración de compilación
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mae', 'mse']
)

# ✅ AGREGAR: Callbacks para mejor entrenamiento
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_mae',
        factor=0.5,
        patience=5,
        min_lr=1e-7
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_mae',
        save_best_only=True
    )
]

# Entrenar el modelo con más epochs y callbacks
print("Iniciando entrenamiento...")
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,  # Más epochs con early stopping
    callbacks=callbacks,
    verbose=1
)

# ✅ CORREGIR: Preparar imágenes de prueba con ruta correcta
# Verificar primero si existe el archivo test_labels.csv o crear lista de imágenes
import os

# Opción A: Si tienes test_labels.csv en sample_data
if os.path.exists('sample_data/test_labels.csv'):
    test_df = pd.read_csv('sample_data/test_labels.csv')
    test_paths = ['test_images/' + fname for fname in test_df['image']]

# Opción B: Si no tienes CSV, leer directamente los archivos de test_images
else:
    test_files = [f for f in os.listdir('test_images/') if f.endswith(('.jpg', '.jpeg', '.png'))]
    test_paths = ['test_images/' + fname for fname in test_files]
    test_df = pd.DataFrame({'image': test_files})

print(f"Encontradas {len(test_paths)} imágenes de prueba")

# Crear dataset de prueba
test_ds = tf.data.Dataset.from_tensor_slices(test_paths)
test_ds = test_ds.map(lambda x: tf.image.resize(
    tf.image.decode_jpeg(tf.io.read_file(x), channels=3), (128,128)) / 255.0).batch(16)

# Predecir los conteos en el conjunto de prueba
print("Generando predicciones...")
predictions = model.predict(test_ds)
predictions = tf.squeeze(predictions).numpy()

# ✅ MEJORAR: Validar predicciones
# Asegurar que no hay valores negativos
predictions = np.maximum(predictions, 0)

# Crear archivo de envío (CSV)
submission = pd.DataFrame({
    'image': test_df['image'],
    'predicted_count': np.round(predictions).astype(int)
})

# Mostrar estadísticas de las predicciones
print(f"Estadísticas de predicciones:")
print(f"Min: {submission['predicted_count'].min()}")
print(f"Max: {submission['predicted_count'].max()}")
print(f"Promedio: {submission['predicted_count'].mean():.2f}")

submission.to_csv('submission.csv', index=False)
print("Archivo submission.csv creado exitosamente!")

# ✅ OPCIONAL: Visualizar algunas predicciones
print("\nPrimeras 10 predicciones:")
print(submission.head(10))